In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import mean_squared_error as mse
from skimage.metrics import structural_similarity as ssim
from PIL import Image

In [13]:
def compress_pca(image, n_components):
    image_array = np.array(image)
    shape = image_array.shape
    image_array = image_array.reshape(-1, shape[-1])
    
    pca = PCA(n_components=n_components)
    compressed = pca.inverse_transform(pca.fit_transform(image_array))
    
    compressed_image = compressed.reshape(shape)
    return Image.fromarray(compressed_image.astype(np.uint8))

def compress_kmeans(image, n_colors):
    image_array = np.array(image)
    shape = image_array.shape
    image_array = image_array.reshape(-1, shape[-1])
    
    kmeans = KMeans(n_clusters=n_colors, random_state=42)
    labels = kmeans.fit_predict(image_array)
    compressed_image = kmeans.cluster_centers_[labels].reshape(shape)
    
    return Image.fromarray(compressed_image.astype(np.uint8))

In [14]:
def calculate_metrics(original_image, compressed_image):
    original_array = np.array(original_image)
    compressed_array = np.array(compressed_image)
    
    win_size = min(original_array.shape[0], original_array.shape[1], 5)  # Adjust the window size here
    
    psnr_value = psnr(original_array, compressed_array)
    mse_value = mse(original_array, compressed_array)
    ssim_value = ssim(original_array, compressed_array, win_size=7, multichannel=True)
    
    return psnr_value, mse_value, ssim_value


In [15]:
def plot_metrics(psnr_values, mse_values, ssim_values, labels):
    plt.figure(figsize=(10, 6))
    
    plt.subplot(1, 3, 1)
    plt.plot(labels, psnr_values, marker='o')
    plt.xlabel('Compression Technique')
    plt.ylabel('PSNR')
    plt.title('PSNR Comparison')

    plt.subplot(1, 3, 2)
    plt.plot(labels, mse_values, marker='o')
    plt.xlabel('Compression Technique')
    plt.ylabel('MSE')
    plt.title('MSE Comparison')

    plt.subplot(1, 3, 3)
    plt.plot(labels, ssim_values, marker='o')
    plt.xlabel('Compression Technique')
    plt.ylabel('SSIM')
    plt.title('SSIM Comparison')

    plt.tight_layout()
    plt.show()

In [16]:
# Load the original image
original_image = Image.open('./image4.jpg')

# Set the number of components for PCA and number of colors for K-means
n_pca_components = 3
n_kmeans_colors = 64

# Compress the image using PCA
compressed_pca = compress_pca(original_image, n_pca_components)

# Compress the image using K-means
compressed_kmeans = compress_kmeans(original_image, n_kmeans_colors)

# Calculate the metrics for both compression techniques
psnr_values = []
mse_values = []
ssim_values = []

c:\Users\ASUS\miniconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [17]:
# Calculate metrics for PCA compression
# psnr_pca, mse_pca, ssim_pca = calculate_metrics(original_image, compressed_pca)
psnr_pca, mse_pca, ssim_pca = calculate_metrics(original_image, Image.open('./PCA/pca_ghyb.jpeg'))
psnr_values.append(psnr_pca)
mse_values.append(mse_pca)
ssim_values.append(ssim_pca)

# Calculate metrics for K-means compression
# psnr_kmeans, mse_kmeans, ssim_kmeans = calculate_metrics(original_image, compressed_kmeans)
psnr_kmeans, mse_kmeans, ssim_kmeans = calculate_metrics(original_image, Image.open('./Kmeans/K_64.jpeg'))
psnr_values.append(psnr_kmeans)
mse_values.append(mse_kmeans)
ssim_values.append(ssim_kmeans)

ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.

In [ ]:
# Create labels for the compression techniques
labels = ['PCA', 'K-means']

# Plot the performance metrics
plot_metrics(psnr_values, mse_values, ssim_values, labels)